# Работа с Datastores и Datasets в Azure ML

## Соединение со своим Workspace

Импорт модулей:

In [ ]:
import os
import azureml.core
from azureml.core import Workspace, Experiment, Dataset
from azureml.core.datastore import Datastore
from azureml.data.data_reference import DataReference
from msrest.exceptions import HttpOperationError

# Check core SDK version number
print(f'SDK version: {azureml.core.VERSION}')

Создаем соединение со своим Workspace:

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, sep = '\n')

## Просмотр существующих Datastore

In [ ]:
# Get the default datastore
default_ds = ws.get_default_datastore()
print(f'Default Datastore name: {default_ds.name}\n')

# Enumerate all datastores, indicating which is the default
for ds_name in ws.datastores:
    print(ds_name, "- Default =", ds_name == default_ds.name)

## Создаем новый Datastore

Установка имени нового Datastore и указание информации об Azure Storage Account, где будет распологаться новый Datastore:

In [ ]:
datastore_name = 'demo_datastore'

# Azure Storage Account Info
storage_account_name = 'aiclouddata'
storage_container_name = 'demo-container'
storage_account_key = '<azure_account_key>' # WARN: insert your storage account key here

Создаем Datastore, если он уже не существует:

In [ ]:
try:
    new_datastore = Datastore.get(ws, datastore_name)
    print(f'Blob Datastore with name {new_datastore.name} was found!')

except HttpOperationError:
    new_datastore = Datastore.register_azure_blob_container(
        workspace=ws,
        datastore_name=datastore_name,
        account_name=storage_account_name,
        container_name=storage_container_name,
        account_key=storage_account_key)
    print(f'Registered blob datastore with name {new_datastore.name}')

Получаем информацию о cозданном Datastore:

In [ ]:
print(f'Datastore {new_datastore.name} based on {new_datastore.datastore_type} in storage account named {new_datastore.account_name}')

## Загрузка данных из Datastore

Делаем созданный Datastore хранилищем по умолчанию (для удобства дальнейшей работы):

In [ ]:
ws.set_default_datastore(new_datastore.name)
ds = ws.get_default_datastore()

print(ds.name)

Загрузка данных:

In [ ]:
ds.upload_files(files=['../data/diabetes_train.csv', '../data/diabetes_test.csv'], # Upload the diabetes csv files in /data
                       target_path='diabetes-data/', # Put it in a folder path in the datastore
                       overwrite=True, # Replace existing files of the same name
                       show_progress=True)

Зарегистрируем загруженные в Datastore данные, как табличный Dataset:

In [ ]:
diabetes_ds = Dataset.Tabular.from_delimited_files(path=(ds, 'diabetes-data/*.csv'))
diabetes_ds

## Зарегистрируем Dataset

In [ ]:
diabetes_db = diabetes_ds.register(workspace = ws,
                                   name = 'diabetes_db',
                                   description = 'Diabetes Disease Database',
                                   create_new_version = True)

Просмотрим список зарегистрированных Наборов данных:

In [ ]:
print('Available datasets:')

for ds in ws.datasets:
    print(f'\t{ds}')

## Просмотр Набора данных

Скачаем зарегистрированный набор данных и выведем 10 строк:

In [ ]:
diabetes_db_from_azure = ws.datasets.get('diabetes_db')

diabetes_df = diabetes_db_from_azure.to_pandas_dataframe()
diabetes_df.sample(10)